Use only the data after 2023


In [ ]:
import pandas as pd
import numpy as np
import warnings

data_path = "data/"

In [119]:
df = pd.read_csv(f"{data_path}chicago_crime.csv")
df.shape[0]

8262433

In [120]:
df["Date"] = pd.to_datetime(df["Date"])
df_after = df[df["Date"].dt.year == 2024]

In [ ]:
df_clean = df_after.copy()
df_clean = df_clean.sort_values(by="Date")
df_clean["Date"] = pd.to_datetime(df_clean["Date"], format="%Y-%m-%dT%H:%M:%S.%f")
df_clean["Year"] = df_clean["Date"].dt.year
df_clean["Month"] = df_clean["Date"].dt.month
df_clean["Day"] = df_clean["Date"].dt.day
df_clean["Hour"] = df_clean["Date"].dt.hour
df_clean["WeekDay"] = df_clean["Date"].dt.weekday
df_clean["IsWeekday"] = (df_clean["WeekDay"] < 5).astype(int)
df_clean = df_clean.dropna(subset=["Longitude", "Latitude"])
df_clean["Location Description"] = df_clean["Location Description"].fillna(
    "OTHER (SPECIFY)"
)

In [122]:
location_mapping = {
    # Residential locations
    "RESIDENTIAL": [
        "APARTMENT",
        "RESIDENCE",
        "RESIDENCE - GARAGE",
        "RESIDENCE - PORCH / HALLWAY",
        "RESIDENCE - YARD (FRONT / BACK)",
        "HOUSE",
        "PORCH",
        "YARD",
        "GARAGE",
        "DRIVEWAY - RESIDENTIAL",
        "DRIVEWAY",
        "CHA APARTMENT",
        "CHA HALLWAY / STAIRWELL / ELEVATOR",
        "CHA HALLWAY",
        "CHA LOBBY",
        "CHA STAIRWELL",
        "CHA GROUNDS",
        "STAIRWELL",
        "BASEMENT",
        "PORCH",
        "HALLWAY",
        "VESTIBULE",
        "GANGWAY",
        "COLLEGE / UNIVERSITY - RESIDENCE HALL",
        "ROOF",
    ],
    # Commercial retail
    "RETAIL": [
        "DEPARTMENT STORE",
        "SMALL RETAIL STORE",
        "GROCERY FOOD STORE",
        "CONVENIENCE STORE",
        "RETAIL STORE",
        "DRUG STORE",
        "APPLIANCE STORE",
        "PAWN SHOP",
        "AUTO / BOAT / RV DEALERSHIP",
        "LIQUOR STORE",
        "TAVERN / LIQUOR STORE",
        "CLEANING STORE",
        "NEWSSTAND",
    ],
    # Food and entertainment
    "FOOD_ENTERTAINMENT": [
        "RESTAURANT",
        "BAR OR TAVERN",
        "TAVERN",
        "MOVIE HOUSE / THEATER",
        "SPORTS ARENA / STADIUM",
        "BOWLING ALLEY",
        "POOL ROOM",
        "CASINO/GAMBLING ESTABLISHMENT",
        "ATHLETIC CLUB",
    ],
    # Transportation
    "TRANSPORTATION": [
        "CTA TRAIN",
        "CTA PLATFORM",
        "CTA BUS",
        "CTA STATION",
        "CTA BUS STOP",
        "CTA TRACKS - RIGHT OF WAY",
        'CTA "L" TRAIN',
        'CTA "L" PLATFORM',
        "CTA SUBWAY STATION",
        "CTA PROPERTY",
        "CTA PARKING LOT / GARAGE / OTHER PROPERTY",
        "TAXICAB",
        "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)",
        "VEHICLE NON-COMMERCIAL",
        "VEHICLE - COMMERCIAL",
        "VEHICLE - DELIVERY TRUCK",
        "VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS",
        "VEHICLE - COMMERCIAL: TROLLEY BUS",
        "OTHER COMMERCIAL TRANSPORTATION",
        "AUTO",
        "OTHER RAILROAD PROPERTY / TRAIN DEPOT",
        "RAILROAD PROPERTY",
    ],
    # Street and outdoor public areas
    "STREET_OUTDOOR": [
        "STREET",
        "SIDEWALK",
        "ALLEY",
        "VACANT LOT / LAND",
        "VACANT LOT",
        "HIGHWAY / EXPRESSWAY",
        "PARKING LOT",
        "BRIDGE",
        "PARK PROPERTY",
        "LAKEFRONT / WATERFRONT / RIVERBANK",
        "BEACH",
        "FOREST PRESERVE",
        "FARM",
    ],
    # Parking and vehicle related
    "PARKING": [
        "PARKING LOT / GARAGE (NON RESIDENTIAL)",
        "GAS STATION",
        "GAS STATION DRIVE/PROP.",
        "CAR WASH",
        "PARKING LOT",
        "CHA PARKING LOT / GROUNDS",
        "CHA PARKING LOT",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "AIRPORT PARKING LOT",
    ],
    # Financial institutions
    "FINANCIAL": [
        "BANK",
        "CURRENCY EXCHANGE",
        "ATM (AUTOMATIC TELLER MACHINE)",
        "CREDIT UNION",
        "SAVINGS AND LOAN",
    ],
    # Office and business
    "OFFICE_BUSINESS": [
        "COMMERCIAL / BUSINESS OFFICE",
        "MEDICAL / DENTAL OFFICE",
        "WAREHOUSE",
        "FACTORY / MANUFACTURING BUILDING",
        "OFFICE",
        "CONSTRUCTION SITE",
        "ANIMAL HOSPITAL",
    ],
    # Educational
    "EDUCATIONAL": [
        "SCHOOL - PUBLIC BUILDING",
        "SCHOOL - PUBLIC GROUNDS",
        "SCHOOL - PRIVATE BUILDING",
        "SCHOOL - PRIVATE GROUNDS",
        "COLLEGE / UNIVERSITY - GROUNDS",
        "LIBRARY",
        "SCHOOL YARD",
        "DAY CARE CENTER",
    ],
    # Government and public services
    "GOVERNMENT": [
        "GOVERNMENT BUILDING / PROPERTY",
        "FEDERAL BUILDING",
        "HOSPITAL BUILDING / GROUNDS",
        "HOSPITAL",
        "FIRE STATION",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "NURSING / RETIREMENT HOME",
        "JAIL / LOCK-UP FACILITY",
    ],
    # Religious
    "RELIGIOUS": ["CHURCH / SYNAGOGUE / PLACE OF WORSHIP"],
    # Airport
    "AIRPORT": [
        "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - SECURE AREA",
        "AIRPORT EXTERIOR - SECURE AREA",
        "AIRPORT EXTERIOR - NON-SECURE AREA",
        "AIRPORT TRANSPORTATION SYSTEM (ATS)",
        "AIRPORT VENDING ESTABLISHMENT",
        "AIRPORT/AIRCRAFT",
        "AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA",
        "AIRCRAFT",
    ],
    # Specialty retail/service
    "SPECIALTY_RETAIL": ["BARBERSHOP", "BARBER SHOP/BEAUTY SALON", "KENNEL"],
    # Water related
    "WATER": ["BOAT / WATERCRAFT"],
    # Abandoned/vacant
    "ABANDONED": ["ABANDONED BUILDING"],
    # Coin operated
    "COIN_OPERATED": ["COIN OPERATED MACHINE"],
    # Cemetery
    "CEMETERY": ["CEMETARY"],
    # Hotel
    "LODGING": ["HOTEL / MOTEL", "HOTEL"],
    "OTHER (SPECIFY)": ["OTHER"],
}

# Create reverse mapping (from specific location to group)
reverse_mapping = {}
for group, locations in location_mapping.items():
    for location in locations:
        reverse_mapping[location] = group


def map_to_location_group(location):
    if location in reverse_mapping:
        return reverse_mapping[location]
    if location != "OTHER (SPECIFY)":
        warnings.warn("Unknown location:" + location)
    return "OTHER"


df_clean.loc[:, "Location Group"] = df_clean.loc[:, "Location Description"].apply(
    map_to_location_group
)
df_clean.loc[:, "Location Group"] = df_clean.loc[:, "Location Description"].apply(
    map_to_location_group
)

In [123]:
df_clean.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Updated On,Latitude,Longitude,Location,Month,Day,Hour,WeekDay,IsWeekday,Location Group
7995931,13325994,JH101383,2024-01-01,036XX S WOLCOTT AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,...,12/21/2024 03:40:46 PM,41.827691,-87.672772,"(41.827691054, -87.672772029)",1,1,0,0,1,STREET_OUTDOOR
7998487,13325413,JH100745,2024-01-01,107XX S EMERALD AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,12/21/2024 03:40:46 PM,41.698826,-87.641371,"(41.698826001, -87.641370983)",1,1,0,0,1,RESIDENTIAL
7994901,13325276,JH100524,2024-01-01,091XX S LAFLIN ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,12/21/2024 03:40:46 PM,41.727774,-87.660400,"(41.727773612, -87.660399528)",1,1,0,0,1,STREET_OUTDOOR
8001130,13325271,JH100317,2024-01-01,013XX S CLAREMONT AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,12/21/2024 03:40:46 PM,41.864510,-87.685162,"(41.864510055, -87.685161602)",1,1,0,0,1,RESIDENTIAL
7994915,13326017,JH101356,2024-01-01,067XX N ASHLAND AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,...,12/21/2024 03:40:46 PM,42.004791,-87.670575,"(42.004790901, -87.670575279)",1,1,0,0,1,RESIDENTIAL


In [124]:
df_clean.nunique()

ID                      257320
Case Number             257291
Date                    128650
Block                    28633
IUCR                       340
Primary Type                31
Description                318
Location Description       128
Arrest                       2
Domestic                     2
Beat                       275
District                    23
Ward                        50
Community Area              77
FBI Code                    26
X Coordinate             50438
Y Coordinate             70604
Year                         1
Updated On                 418
Latitude                122651
Longitude               122644
Location                122676
Month                       12
Day                         31
Hour                        24
WeekDay                      7
IsWeekday                    2
Location Group              19
dtype: int64

In [ ]:
cutoff_idx = int(len(df_clean) * 0.8)
cutoff_date = df_clean.iloc[cutoff_idx]["Date"]

# Split based on date
df_train = df_clean[df_clean["Date"] < cutoff_date].copy()
df_test = df_clean[df_clean["Date"] >= cutoff_date].copy()
impute_values = {
    "Ward": df_train["Ward"].mode()[0],
    "Community Area": df_train["Community Area"].mode()[0],
}
for column, value in impute_values.items():
    df_train[column] = df_train[column].fillna(value)
    df_test[column] = df_test[column].fillna(value)

Data conversion


Handle missing values:\
For longitude and latitude, fill with median of the block or district.
For ward and community area, fill with the most frequent value in the same district.


Transform date to datetime format and extract other date-related data


Aggregate for spatial data


Transform boolean data to integer


In [126]:
df_train_sorted_lat = df_train["Latitude"].sort_values()
lat_bin_edges = np.linspace(
    df_train_sorted_lat.min(), df_train_sorted_lat.max(), 51
)  # 51 edges for 50 bins
df_train["lat_bin"] = pd.cut(df_train["Latitude"], bins=lat_bin_edges)
df_test["lat_bin"] = pd.cut(df_test["Latitude"], bins=lat_bin_edges)
df_train_sorted_lon = df_train["Longitude"].sort_values()
lon_bin_edges = np.linspace(
    df_train_sorted_lon.min(), df_train_sorted_lon.max(), 51
)  # 51 edges for 50 bins
df_train["lon_bin"] = pd.cut(df_train["Longitude"], bins=lon_bin_edges)
df_test["lon_bin"] = pd.cut(df_test["Longitude"], bins=lon_bin_edges)

In [127]:
# Create time of day and season features
hour_bins = [0, 6, 12, 18, 24]
hour_labels = ["Night", "Morning", "Afternoon", "Evening"]
season_bins = [0, 3, 6, 9, 12]
season_labels = ["Winter", "Spring", "Summer", "Fall"]

df_train["TimeCategory"] = pd.cut(df_train["Hour"], bins=hour_bins, labels=hour_labels)
df_test["TimeCategory"] = pd.cut(df_test["Hour"], bins=hour_bins, labels=hour_labels)
df_train["Season"] = pd.cut(df_train["Month"], bins=season_bins, labels=season_labels)
df_test["Season"] = pd.cut(df_test["Month"], bins=season_bins, labels=season_labels)

In [128]:
location_counts = df_train["Location Description"].value_counts()
threshold = len(df_train) * 0.01  # Keep categories that appear in at least 1% of data
keep_locations = location_counts[location_counts >= threshold].index


def map_rare_categories(df, column, keep_values):
    df_copy = df.copy()
    df_copy.loc[~df_copy[column].isin(keep_values), column] = "Other"
    return df_copy


df_train = map_rare_categories(df_train, "Location Description", keep_locations)
df_test = map_rare_categories(df_test, "Location Description", keep_locations)
df_train.to_csv(f"{data_path}processed_train_data.csv", index=False)
df_test.to_csv(f"{data_path}processed_test_data.csv", index=False)

In [132]:
df_train.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Month,Day,Hour,WeekDay,IsWeekday,Location Group,lat_bin,lon_bin,TimeCategory,Season
8202767,13630381,JH467941,2024-10-14 07:00:00,080XX S MARYLAND AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,...,10,14,7,0,1,RESIDENTIAL,"(41.743, 41.75]","(-87.607, -87.598]",Morning,Fall
8200840,13630252,JH467597,2024-10-14 07:04:00,070XX S COTTAGE GROVE AVE,051A,ASSAULT,AGGRAVATED - HANDGUN,STREET,True,False,...,10,14,7,0,1,STREET_OUTDOOR,"(41.766, 41.773]","(-87.607, -87.598]",Morning,Fall
8202496,13630210,JH467585,2024-10-14 07:09:00,041XX W GRAND AVE,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,Other,False,False,...,10,14,7,0,1,RETAIL,"(41.902, 41.909]","(-87.73, -87.721]",Morning,Fall
8203177,13630513,JH468161,2024-10-14 07:10:00,024XX N MC VICKER AVE,0710,THEFT,THEFT FROM MOTOR VEHICLE,STREET,False,False,...,10,14,7,0,1,STREET_OUTDOOR,"(41.924, 41.932]","(-87.779, -87.771]",Morning,Fall
8202551,13630131,JH467576,2024-10-14 07:10:00,008XX W ADDISON ST,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,...,10,14,7,0,1,PARKING,"(41.947, 41.955]","(-87.656, -87.648]",Morning,Fall


In [130]:
df_test.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Month,Day,Hour,WeekDay,IsWeekday,Location Group,lat_bin,lon_bin,TimeCategory,Season
8199681,13630180,JH467641,2024-10-14 07:14:00,065XX S DR MARTIN LUTHER KING JR DR,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,True,False,...,10,14,7,0,1,RESIDENTIAL,"(41.773, 41.781]","(-87.623, -87.615]",Morning,Fall
8200674,13630186,JH467693,2024-10-14 07:14:00,003XX W 80TH ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,False,...,10,14,7,0,1,RESIDENTIAL,"(41.743, 41.75]","(-87.639, -87.631]",Morning,Fall
8204956,13632225,JH470197,2024-10-14 07:17:00,075XX S EXCHANGE AVE,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,...,10,14,7,0,1,PARKING,"(41.758, 41.766]","(-87.566, -87.557]",Morning,Fall
8200248,13630198,JH467680,2024-10-14 07:30:00,024XX S RIDGEWAY AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,10,14,7,0,1,STREET_OUTDOOR,"(41.841, 41.849]","(-87.721, -87.713]",Morning,Fall
8202041,13630181,JH467636,2024-10-14 07:33:00,075XX N ASHLAND AVE,1477,WEAPONS VIOLATION,RECKLESS FIREARM DISCHARGE,STREET,False,False,...,10,14,7,0,1,STREET_OUTDOOR,"(42.015, 42.023]","(-87.672, -87.664]",Morning,Fall


In [131]:
df_train.shape

(205855, 32)